In [19]:
import sqlite3
import pandas as pd
con = sqlite3.connect('d19_day15.sqlite')

def x(q):
    return pd.read_sql_query(q,con)

In [20]:
#How many people actually provided a review?
q = 'select count(*) from rating'
x(q).head()

,count(*)
0,359


In [26]:
q = 'select distinct count(rid) from rating'
x(q)

,count(rid)
0,359


In [23]:
# Who signed up as a reviewer but did not rate any movie?
signed_up = 'select rid, firstname, lastname from reviewer'
did_reviews = 'select distinct rid from rating'
q = 'select rid from (' + signed_up + ') where rid not in (' + did_reviews + ')'
q2 = 'select firstname, lastname from (' + q + ') natural join reviewer'

x(signed_up).head()

,rid,firstname,lastname
0,777,John,Ringland
1,102,Tong,Wu
2,3333,Chima,Ezeilo
3,716,Ashok,Kumar
4,515,Ji Ho,Lee


In [28]:
q = 'select firstname, lastname from rating natural join movie natural join reviewer where title="Avatar"'
x(q)

,firstname,lastname
0,Abhishek,Mishra
1,Abhishek,Mishra
2,Abhishek,Mishra
3,Abhishek,Mishra
4,Dorien,Verbruggen
5,Dorien,Verbruggen
6,Dorien,Verbruggen
7,Dorien,Verbruggen
8,Eugen,Feng
9,Eugen,Feng


In [31]:
# Which movies were rated by more than one person?
#NOTE: use of "having"
q = 'select count(rid) as nrev, mid from rating group by mid'
q2 = 'select title, nrev from (' + q + ') natural join movie'
x(q2)

,title,nrev
0,Birdman,8
1,The Incredibles,6
2,City Hall,2
3,Topsy Turvy,2
4,Avengers: EndGame,10
...,...,...
214,Amelie,2
215,Chicago,2
216,The Chorus,2
217,Call Me by Your Name,2


In [32]:
q = 'select count(rid) as nrev, mid from rating group by mid having nrev>1'
q2 = 'select title,nrev from (' + q + ') natural join movie'
x(q2).head()

,title,nrev
0,Birdman,8
1,The Incredibles,6
2,City Hall,2
3,Topsy Turvy,2
4,Avengers: EndGame,10


In [44]:
# Who is the best director?
# i.e. what is the average of all the ratings of movies directed by each director

# first get a table of average ratings grouped by director 
q = 'select avg(stars) as avstars, director from rating natural join movie group by director having director != ""'
q2 = 'select * from (' + q + ') where avstars=5'
x(q2).head()

,avstars,director
0,5.0,Akiro Kurosawa
1,5.0,Ang Lee
2,5.0,Bong Joon-ho
3,5.0,Christophe Barratier
4,5.0,Christopher McQuarrie


In [43]:

q2 = 'select * from (' + q + ') where avstars=(select max(avstars) from ('+ q +'))'
x(q2).head()


,avstars,director
0,5.0,Akiro Kurosawa
1,5.0,Ang Lee
2,5.0,Bong Joon-ho
3,5.0,Christophe Barratier
4,5.0,Christopher McQuarrie


In [45]:
def extremizers( q, agg, field ):
    return 'select * from (' + q + ') where ' + field + '= (select ' + agg + '(' + field + ') from (' + q + '))'
extremizers( 'movie', 'max', 'year' )

'select * from (movie) where year= (select max(year) from (movie))'

In [46]:
q = 'select avg(stars) as avstars, director from rating natural join movie group by director having director != ""'
x( extremizers( q, 'max', 'avstars') ).head()

,avstars,director
0,5.0,Akiro Kurosawa
1,5.0,Ang Lee
2,5.0,Bong Joon-ho
3,5.0,Christophe Barratier
4,5.0,Christopher McQuarrie


In [55]:
# Of the directors with the highest avg rating, which have the largest number of ratings?

q = 'select avg(stars) as avstars, count(stars) as nrev, director from rating natural join movie group by director having director != ""'
q2 = extremizers( q, 'max', 'avstars' )
x(q2).head()

,avstars,nrev,director
0,5.0,4,Akiro Kurosawa
1,5.0,4,Ang Lee
2,5.0,4,Bong Joon-ho
3,5.0,4,Christophe Barratier
4,5.0,4,Christopher McQuarrie


# SQL "with" construct

In [59]:
q = 'select avg(stars) as avstars, director from rating natural join movie group by director having director != ""'
q2 = 'select * from (' + q + ') where avstars=(select max(avstars) from (' + q + '))'as

q2 = 'with QQQ as (' + q + ') select * from QQQ where avstars = (select max(avstars) from QQQ)'
print(q2)
x(q2)

with QQQ as (select avg(stars) as avstars, director from rating natural join movie group by director having director != "") select * from QQQ where avstars = (select max(avstars) from QQQ)


,avstars,director
0,5.0,Akiro Kurosawa
1,5.0,Ang Lee
2,5.0,Bong Joon-ho
3,5.0,Christophe Barratier
4,5.0,Christopher McQuarrie
5,5.0,Edward Yang
6,5.0,Gavin O'Connor
7,5.0,Giuseppe Tornatore
8,5.0,Gregory Hoblit
9,5.0,Hayao Miyazaki
